In [1]:
import os
import torch
import numpy as np

import wandb
import datetime
from tqdm.auto import tqdm

from timm.models import resnet18
from torchsummary import summary

In [2]:
model = resnet18(pretrained=True).cuda()

summary(model, (3, 128, 128))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 64, 64]           9,408
       BatchNorm2d-2           [-1, 64, 64, 64]             128
              ReLU-3           [-1, 64, 64, 64]               0
         MaxPool2d-4           [-1, 64, 32, 32]               0
            Conv2d-5           [-1, 64, 32, 32]          36,864
       BatchNorm2d-6           [-1, 64, 32, 32]             128
          Identity-7           [-1, 64, 32, 32]               0
              ReLU-8           [-1, 64, 32, 32]               0
          Identity-9           [-1, 64, 32, 32]               0
           Conv2d-10           [-1, 64, 32, 32]          36,864
      BatchNorm2d-11           [-1, 64, 32, 32]             128
             ReLU-12           [-1, 64, 32, 32]               0
       BasicBlock-13           [-1, 64, 32, 32]               0
           Conv2d-14           [-1, 64,

In [ ]:
from argparse import ArgumentParser
import random

def parse_args():
    parser = ArgumentParser()
    
    parser.add_argument(
        "--dataset_path",
        type=str,
        default="/opt/ml/dataset",
    )
    parser.add_argument(
        "--save_path",
        type=str,
        default="/opt/ml/level3_cv_finalproject-cv-01/model/save",
    )
    parser.add_argument(
        "--model_name",
        type=str,
        default="resnet18",
    )
    parser.add_argument(
        "--seed",
        type=int,
        default=42
    )
    parser.add_argument(
        "--num_workers",
        type=int,
        default=8
    )
    parser.add_argument(
        "--epochs",
        type=int,
        default=50
    )
    parser.add_argument(
        "--batch",
        type=int,
        default=16
    )
    parser.add_argument(
        "--accumulation_steps",
        type=int,
        default=4
    )
    parser.add_argument(
        "--lr",
        type=float,
        default=1e-3
    )
    parser.add_argument(
        "--weight_decay",
        type=float,
        default=1e-3
    )
    
    args = parser.parse_args()
    return args

def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)

In [ ]:
import albumentations as A

def main(args):
    '''
    args : 기본적으로 가져와야할 매개변수를 argParser로 가져온다.
        dataset_path
        model_name
        seed
        num_workers
        epochs
        batch
        lr
        weight_decay
    '''
    torch.cuda.empty_cache()
    set_seed(args.seed)
    wandb.init(project="final_project", name=f"cls_{args.model_name}")

    #TODO: set Augmentation
    tf = A.Compose(
        [
            A.resize(512, 512),
            A.normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),# Image net참고, 0을 기준으로 1의 표준편차
        ]
    )
    #TODO: Load Dataset
    #TODO: Load Model
    #TODO: Optimizer & LR Scheduler
    #TODO: Train & Valid

    wandb.finish()

In [ ]:
from sklearn.metrics import f1_score

def validation(model, criterion, valid_loader):
    print("Start Validation...")

    total_loss, scores = [], []
    for images, labels in tqdm(valid_loader, total=len(valid_loader)):
        image, label = image.cuda(), label.cuda()
        model.cuda()

        with torch.cuda.amp.autocast_mode(enabled=True):
            outputs = model(images)
            loss = criterion(outputs, label)
        
        score = f1_score(y_true=labels, y_pred=outputs, average='macro')
        total_loss.append(loss)
        scores.append(score)
    #TODO : total_loss, scores 평균
    return mean_loss, mean_score

In [ ]:
def train(model, optimizer, criterion, scheduler, train_loader, valid_loader, args):
    print("Start Train...")

    scaler = torch.cuda.amp.GradScaler(enabled=True)
    cur_time = datetime.datetime.now()
    best_score = 0
    for epoch in range(args.epochs):
        model.train()
        optimizer.zero_grad()
        for step, (images, labels) in enumerate(train_loader):
            image, label = images.cuda(), labels.cuda()
            model.cuda()

            with torch.cuda.amp.autocast_mode(enabled=True):
                outputs = model(image)
                loss = criterion(outputs, label)
            
            scaler.scale(loss).backward()
            if not (step + 1) % args.accumulation_steps:
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad()

            wandb.log({"train/LR": args.lr, "train/loss": loss})
            if not (step+1) % 20:
                f'Duration :{cur_time - datetime.datetime.now()} | '
                f"Epoch [{epoch+1}/{args.epochs}], "
                f"Step [{step+1}/{len(train_loader)}], "
                f"Loss: {round(loss.item(),4)}"
                cur_time = datetime.datetime.now()

        wandb.log({"train/Epoch": epoch+1})
        scheduler.step()

        if not (epoch+1) % 10:
            loss, score = validation(model, criterion, valid_loader, args)

            print(f"Validation Marco-F1 Score : {score}")
            wandb.log({"val/Loss": loss, "val/Score": score})

            if score > best_score:
                best_score = score
                output_path = os.path.join(args.save_path, f"{args.model_name}_best.pth")
                torch.save(model, output_path)
            output_path = os.path.join(args.save_path, f"{args.model_name}_latest.pth")
            torch.save(model, output_path)
    print("Finish Train!!!")